## project starts from here!

In [ ]:
import matplotlib.pyplot as plt
# make sure plots are displayed correctly
%matplotlib inline
import numpy as np
from scipy import linalg
from mpl_toolkits import mplot3d

# Generating weights
From the lecture slide, we know that input-to-reservoir weight $Win$ and reservoir weight $Wr$ are drawn independently from a uniform distribution in $[-1,1]$.
As $Wr$ directly impacts on the dynamics of the network. I follow the first rule of the lecture for generating $Wr$.

$Wr = a \frac{Wr}{\rho{(Wr)}}$

Here, $\rho{(Wr)}$ is the Spectral radius which is the largest absolute value of its eigenvalues. $a$ is a scaler hyper-parameter. Here, a is evaluated in range 0.1 to 1.1.

In [ ]:
def generate_weights(a, inshape, reservior):
    np.random.seed(62)
    Win = (np.random.rand(reservior, inshape) - 0.5) * 1
    Wr = np.random.rand(reservior, reservior) - 0.5
    #a = 0.99
    print('Computing spectral radius...')
    rhoW = max(abs(linalg.eig(Wr)[0]))
    print('done.')
    print(rhoW)
    Wr *= a / rhoW

    return Win, Wr

# Define the Loss

In [ ]:
def MSE(label,pred):
    loss = np.square(np.subtract(label,pred)).mean()
    return loss

# ESN Network
1. creating the input-output pair. let t be the input time series. I have created a output pair y=t+1. Thus, the input-output pair is ${(t,y)}_{i=1}^{N-1}$.

2. Split the data into train_pair and test_pair, where ${(train\_pair)}_{1}^{N-1-K}$ and ${(test\_pair)}_{N-1-K}^{N-1}$.

3. Genarate the input-to-reservoir weight  $Win$  and reservoir weight $Wr$ using the method discussed above.

4. Create matrix $X \in R^{N \times N_r}$, where, $N=N-1$ and $N_r = $size of the reservior.

5. Intialize the first state x=0

6. Create input matrix $out = R^{len(train\_pair)}$

6. For each t, update the state $x_i$ as follows:
$x_t = tanh(Wr x_{t-1} + Win t)$
then collect all the resulting states in $X$.

7. Calculate the Wout using least-square problem as follows:
$Wout = (X^T X)^{-1}  X^T out$ as the inverse operation takes alot of computation resource. I have used scipy.linalg.solve(). Thus, the equation becomes
$ (X X^T) Wout = (X out^{T})^T$

these steps are upto the training phase and finding out Wout.

Here begins the k-step prediction.

8. Create prediction matrix $Y \in R^K$.

9. Initialize first time step $t=train\_pair[-1,1]$ i.e. last time-step data of the training set. The subsequent data will be predicted by the model.

10. For each t, update the state $x_i$ as follows:
$x_t = tanh(Wr x_{t-1} + Win t)$ then
$ y = Wout x_t$.  Finally collect all the prediction in $Y$.

11. Calculate the mean square loss between the ground truth time step and predicted time step.

In [ ]:
def ESN(name, reservior = 300, k = 20, a=0.99):
    inx = 1
    outx = 1
    
    # creating the input output data
    data = np.loadtxt(str(name)+'.txt')

    iopair = np.zeros((len(data) - 1, 2), dtype=float)

    for i in range(len(data) - 1):
        iopair[i, 0] = data[i]
        iopair[i, 1] = data[i + 1]

    # k step prediction
    split = iopair.shape[0] - k
    train_pair = iopair[:split, :]
    test_pair = iopair[split:, :]
    plot_label = data[split:]

    # generate the random weights
    Win, Wr = generate_weights(a, inx, reservior)


    # resulting N states
    X = np.zeros((train_pair.shape[0], reservior))

    # set the target matrix
    out = np.array(train_pair[:, 1])
    out = np.expand_dims(out, axis=0)

    # initialize the first state
    x = np.zeros((reservior, 1))

    # ESN layer
    for t in range(train_pair.shape[0]):
        u = np.array(train_pair[t, 0])
        x = np.tanh(np.dot(Win, u) + np.dot(Wr, x))
        X[t, :] = x.squeeze()

    # calculating Wout
    X = X.T
    Wout = linalg.solve(np.dot(X, X.T), np.dot(X, out.T)).T

    Y = np.zeros((outx,k))
    u = np.array(train_pair[-1,1])
    for t in range(test_pair.shape[0]):
        x = np.tanh( np.dot( Win,u) + np.dot( Wr, x ))
        y = np.dot( Wout, x )
        Y[:,t] = y
        u = y

    label =test_pair[:,1]
    label = np.expand_dims(label, axis=0)

    loss = MSE(label,Y)
    
    return loss, plot_label, Y, X.T

# All plots function

In [ ]:
def plot_3d(name, out, k, reservior, a):
    
    x = out[:,0]
    y = out[:,1]
    z = out[:,2]
    
    # Creating figure
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    
    ax.scatter3D(x, y, z, cmap='Greens')

    # Creating plot
    #ax.scatter3D(x, y, z, color="green")
    plt.title(name + " for k= " + str(k) + " state = " +str(reservior) + " a= " + str(a))

    # show plot
    plt.show()

In [ ]:
def plot_prediction(name, label, prediction,k, reservior, a):
    plt.figure(10).clear()
    plt.plot( label.T, 'g' )
    plt.plot( prediction.T, 'b' )
    plt.title(name + ' prediction for k = ' + str(k) + " state = " +str(reservior) + " a = " + str(a))
    plt.legend(['label signal', 'predicted signal'])

# PCA Implementation

In [ ]:
def PCA(X, num_components):
    # Remove mean from the data
    X_remove = X - np.mean(X, axis=0)

    # compute sample convariance matrix
    cov_mat = np.cov(X_remove, rowvar=False)

    # compute eigen decomposition
    eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)

    # sort the eigen values in decreaing order
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:, sorted_index]

    # select the number of components to be preserved
    eigenvector_subset = sorted_eigenvectors[:, 0:num_components]

    # project into selected components
    out = np.dot(eigenvector_subset.transpose(), X_remove.transpose()).transpose()

    return out

# Experiment Section

In [ ]:
# plot of data for lorentz
data = np.loadtxt('lorentz.txt')
# plot some of it
plt.figure(10).clear()
plt.plot(data[15000:])
plt.title('A sample of data of lorentz')

In [ ]:
# plot of data for lorentz
data = np.loadtxt('2sin.txt')
# plot some of it
plt.figure(10).clear()
plt.plot(data[1000:2000])
plt.title('A sample of data of 2sin')

# Experiment on Lorentz

In [ ]:
name = 'lorentz'
k_all = [5, 15, 25, 35, 40]
reservior_all = [100, 200, 300, 400, 500, 600, 700, 800 ]
a_all = [0.1, 0.2, 0.3, 0.4, 0.5, 0.60, 0.7, 0.8, 0.9, 1, 1.10, 1.2, 1.3]

for k in k_all:
    for reservior in reservior_all:
        for a in a_all:
            loss, label, prediction, X = ESN(name, reservior, k, a)
            print("loss at K = " + str(k) + " state = " +str(reservior) + " a = " + str(a) + " -->" + str(loss))
            plot_prediction(name, label, prediction, k, reservior, a)
            out = PCA(X,3)
            plot_3d('PCA', out, k, reservior, a)

# Result Analysis of Lorentz

1. The first obersvation is that we get a good reconstruction of lorentz attractor around the value of $a \simeq  1$ which is expected according to the theoritical knowledge of the lecture.

2. By changing the hyper-parameters, the reconstruction changes. For fixed k and a, the changes of reservior size changes the reconstruction. This is applicable for fixed reservior and a and fixed reservior and k.

3. It is also evident that the reconstruction is stable under small perturbations of the hyper-parameters and random initialization of the developed ESN model. For k =  5 and reservior = 700, the reconstruction is stable for $a = 1$ to $a=1.1$.

4. Given the proper hyper-parameter settings, the ESN model can perfectly predict the choatic time series as it able to predict $k = 40$ time steps for $ reservior = 700 $ and $ a =1 $ as well as can learn the embedding provided by the lorentz attractor visualization. 

# Experiment on 2sin

In [ ]:
name = '2sin'
k_all = [5, 15, 25, 35, 40]
reservior_all = [100, 200, 300, 400, 500 ]
a_all = [ 0.5, 0.60, 0.7, 0.8, 0.9, 1, 1.10, 1.2, 1.3]

for k in k_all:
    for reservior in reservior_all:
        for a in a_all:
            loss, label, prediction, X = ESN(name, reservior, k, a)
            print("loss at K = " + str(k) + " state = " +str(reservior) + " a = " + str(a) + " -->" + str(loss))
            plot_prediction(name, label, prediction, k, reservior, a)
            out = PCA(X,3)
            plot_3d('PCA', out, k, reservior, a)

# Result Analysis of 2sin
The analysis is same as the lorentz. Although the instruction doesn't tell to show the principle components of the 2sin. I visualize to see the embedding pattern. The model can perpectly predict 40 timesteps of 2sin signal at $reservior = 400$ and $ a = 1.3$


# Extra Analysis
I just compare with the LLE because of low computation. The construction very bad as expected because LLE tries to preserve local information. For compare with other dimentionality reduction methods, run the last block of the code on heavy computer.

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
embedding = LocallyLinearEmbedding(n_components=3)

In [ ]:
name = 'lorentz'
reservior = 700
k = 40
a = 1
loss, label, prediction, X = ESN(name, reservior, k, a)
print("loss at K = " + str(k) + " state = " +str(reservior) + " a = " + str(a) + " -->" + str(loss))
plot_prediction(name, label, prediction, k, reservior, a)
out = PCA(X,3)
plot_3d('PCA', out, k, reservior, a)

out_LLE = embedding.fit_transform(X)
plot_3d('LLE', out_LLE, k, reservior, a)

In [ ]:
from sklearn.decomposition import KernelPCA
KPCA = KernelPCA(n_components=3, kernel='linear')
out_KPCA = KPCA.fit_transform(X)
plot_3d('KPCA', out_KPCA, k, reservior, a)

from sklearn.manifold import Isomap
isomap = Isomap(n_components=3)
out_isomap = isomap.fit_transform(X)
plot_3d('ISOMAP', out_isomap, k, reservior, a)

from sklearn.manifold import MDS
mds = MDS(n_components=3)
out_mds = embedding.fit_transform(X)
plot_3d('MDS', out_mds, k, reservior, a)

from sklearn.manifold import TSNE
out_tsne = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(X)
plot_3d('TSNE', out_tsne, k, reservior, a)